<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=778068ddf92dbb7f16ae63f2553a90fd46272c42530360d7828ee053eccc3158
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-21 13:01:41
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.44 C
-------------------
Today PnL: -98.72 K (-0.68%)
Current PnL: -22.17 L (-14.53%)
CY Booked + Current PnL: -8.28 L (-5.43%)
-------------------
Total profit:  1.64 L
Total loss:  -23.81 L
-------------------
Total Booked + Current PnL: 18.70 L (14.88%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.67%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.59 L (60.99%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 7.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.14,-14.11,16.53,0.09,14308.0,-14219.0,86558.0,100.85,50.0,M-SC,4.15,253.0,-0.99,0.60,13.72,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-2.15,-9.16,10.11,0.02,16031.0,-15994.0,158565.0,141.63,70.0,M-SC,4.20,234.0,-1.00,1.10,37.92,OX40N,NTT,PAINTS
83,VOLTAS,1530.00,-1.33,8.97,9.83,19.69,20540.0,17208.0,208950.0,-1.63,54.0,X-MC,3.57,78.0,0.84,1.45,16.38,XY25,NTT,AC
53,NESTLEIND,1377.00,0.56,13.41,7.04,21.40,21192.0,35592.0,301018.0,5.18,65.0,X-LC,7.98,12.0,1.68,2.10,20.94,XY25,NTT,FMCG
35,ICICIGI,2252.93,-0.66,7.99,11.31,20.19,21290.0,13921.0,188241.0,-15.31,54.0,X-MC,7.71,68.0,0.65,1.31,24.31,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,216.0,2.67,-20.21,54.20,23.03,77442.0,-36189.0,142882.0,55.70,47.0,H-SC,11.65,167.0,-0.47,0.99,41.77,XR,NTT,BANKS
40,INDUSINDBK,1800.0,2.54,-37.21,111.64,32.89,54121.0,-28728.0,48478.0,-698.49,65.0,L-MC,5.39,259.0,-0.53,0.34,33.51,XR,NTT,BANKS
67,SIS,528.0,1.15,-24.95,61.57,21.26,51309.0,-27698.0,83334.0,1964.77,40.0,H-SC,3.30,166.0,-0.54,0.58,12.59,OX40N,NTT,MISC
52,MEDANTA,1486.0,1.03,2.33,21.25,24.08,27607.0,2964.0,129914.0,-5.97,43.0,X-SC,4.51,89.0,0.11,0.90,22.41,XY24,NTT,HEALTHCARE
76,TMPV,600.0,0.71,-14.14,65.59,42.18,155671.0,-39071.0,237339.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.71,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-2.91,-14.96,80.50,53.50,206569.0,-45143.0,256608.0,-58.78,47.0,X-MC,3.84,58.0,-0.22,1.79,13.98,XY24,NTT,FMCG
72,TANLA,1943.92,-2.85,-34.34,234.58,119.69,417050.0,-92975.0,177786.0,-33.88,31.0,H-SC,12.49,156.0,-0.22,1.24,40.45,AR,ATH,IT
69,STARHEALTH,761.00,-2.61,-4.73,54.56,47.26,146132.0,-13285.0,267838.0,30.06,50.0,H-SC,15.62,171.0,-0.09,1.87,43.50,XY24,NTT,INSURANCE
0,5PAISA,593.00,-2.55,-33.84,70.65,12.90,102132.0,-73936.0,144560.0,112.74,62.0,H-SC,15.05,161.0,-0.72,1.01,20.20,OX40N,NTT,FINANCE
56,QUESS,424.00,-2.53,-29.14,101.59,42.84,46794.0,-18944.0,46062.0,-53.72,36.0,X-SC,37.16,83.0,-0.40,0.32,0.00,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.26,0.74,71.0,72.27,121201.0,1258.0,170706.0,-13.34,54.0,M-LC,5.03,99.0,0.01,1.19,6.96,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.76,-3.53,115.53,107.91,164548.0,-5215.0,142429.0,-21.36,37.0,M-SC,10.18,216.0,-0.03,0.99,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-2.15,-9.16,10.11,0.02,16031.0,-15994.0,158565.0,141.63,70.0,M-SC,4.20,234.0,-1.00,1.10,37.92,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.14,-14.11,16.53,0.09,14308.0,-14219.0,86558.0,100.85,50.0,M-SC,4.15,253.0,-0.99,0.60,13.72,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.28,-21.00,43.64,13.47,92966.0,-56637.0,213030.0,-67.53,31.0,H-SC,1.67,158.0,-0.61,1.48,8.32,XY24,NTT,PAINTS
44,JCHAC,2282.00,-0.10,-23.23,30.25,-0.01,29679.0,-29693.0,98112.0,19986.96,63.0,M-SC,1.54,233.0,-1.00,0.68,11.10,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.26,0.74,71.0,72.27,121201.0,1258.0,170706.0,-13.34,54.0,M-LC,5.03,99.0,0.01,1.19,6.96,XR,NTT,IT
38,INDIAMART,4810.62,-1.45,1.29,104.1,106.72,130043.0,1585.0,124921.0,-49.87,37.0,H-SC,4.45,139.0,0.01,0.87,24.96,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.45,1.29,104.10,106.72,130043.0,1585.0,124921.0,-49.87,37.0,H-SC,4.45,139.0,0.01,0.87,24.96,AR,ATH,MISC
85,WIPRO,420.00,-0.26,0.74,71.00,72.27,121201.0,1258.0,170706.0,-13.34,54.0,M-LC,5.03,99.0,0.01,1.19,6.96,XR,NTT,IT
25,FINCABLES,1641.55,-0.76,-3.53,115.53,107.91,164548.0,-5215.0,142429.0,-21.36,37.0,M-SC,10.18,216.0,-0.03,0.99,0.00,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,-0.23,-2.37,38.92,35.63,79635.0,-4961.0,204613.0,-16.30,32.0,H-MC,4.77,119.0,-0.06,1.42,13.91,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,-0.66,-5.43,92.36,81.92,100274.0,-6229.0,108569.0,-44.55,31.0,M-SC,6.18,236.0,-0.06,0.76,28.53,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.93,-35.49,109.60,35.20,90968.0,-45670.0,83000.0,8.88,22.0,X-SC,12.02,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,600.00,0.71,-14.14,65.59,42.18,155671.0,-39071.0,237339.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.71,XY24,NTT,AUTO
34,HONAUT,58357.33,-0.25,-15.03,68.66,43.32,95025.0,-24472.0,138400.0,-26.95,26.0,X-SC,1.66,90.0,-0.26,0.96,6.27,X40N,ATH,ELECTRICAL
46,JSWINFRA,342.00,-1.18,-8.02,25.27,15.23,46566.0,-16072.0,184275.0,-21.24,27.0,X-MC,6.35,66.0,-0.35,1.28,21.64,X40N,NTT,REALTY
26,GILLETTE,11206.78,-1.62,-2.31,35.54,32.42,88153.0,-5856.0,248040.0,-19.73,29.0,X-SC,6.30,85.0,-0.07,1.73,12.44,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,0.71,-14.14,65.59,42.18,155671.0,-39071.0,237339.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.71,XY24,NTT,AUTO
43,ITC,452.00,0.62,-0.32,10.80,10.44,21545.0,-650.0,199488.0,-40.34,50.0,X-LC,0.96,5.0,-0.03,1.39,5.22,X40,NTT,FMCG
36,ICICIPRULI,790.00,-0.17,2.35,27.97,30.97,50593.0,4146.0,180884.0,-21.72,54.0,X-MC,1.52,75.0,0.08,1.26,15.20,X40,ATH,INSURANCE
34,HONAUT,58357.33,-0.25,-15.03,68.66,43.32,95025.0,-24472.0,138400.0,-26.95,26.0,X-SC,1.66,90.0,-0.26,0.96,6.27,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-0.62,-22.67,112.28,64.15,206595.0,-53951.0,184000.0,-54.57,47.0,X-SC,1.82,82.0,-0.26,1.28,3.21,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.93,-35.49,109.60,35.20,90968.0,-45670.0,83000.0,8.88,22.0,X-SC,12.02,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.63,-13.51,71.65,48.45,124251.0,-27092.0,173414.0,-19.66,30.0,X-MC,7.28,64.0,-0.22,1.21,0.00,X40N,ATH,FINANCE
56,QUESS,424.00,-2.53,-29.14,101.59,42.84,46794.0,-18944.0,46062.0,-53.72,36.0,X-SC,37.16,83.0,-0.40,0.32,0.00,XY24,NTT,MISC
54,PAGEIND,51605.08,0.61,-5.22,33.00,26.07,51216.0,-8540.0,155200.0,-30.18,34.0,X-MC,8.70,55.0,-0.17,1.08,0.61,X40,ATH,APPARELS
76,TMPV,600.00,0.71,-14.14,65.59,42.18,155671.0,-39071.0,237339.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.71,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-2.53,-29.14,101.59,42.84,46794.0,-18944.0,46062.0,-53.72,36.0,X-SC,37.16,83.0,-0.40,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-0.95,-46.86,186.62,52.31,143951.0,-68024.0,77136.0,-43.47,31.0,X-SC,3.43,91.0,-0.47,0.54,3.25,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.93,-35.49,109.60,35.20,90968.0,-45670.0,83000.0,8.88,22.0,X-SC,12.02,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,1.03,2.33,21.25,24.08,27607.0,2964.0,129914.0,-5.97,43.0,X-SC,4.51,89.0,0.11,0.90,22.41,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-1.51,-3.14,25.53,21.58,34654.0,-4401.0,135738.0,-51.15,37.0,X-MC,7.19,56.0,-0.13,0.95,20.17,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.41,-14.20,39.02,19.28,115821.0,-49132.0,296824.0,-25.24,64.0,X-LC,7.77,1.0,-0.42,2.07,9.73,X40,ATH,IT
41,INFY,2275.00,-0.30,4.60,48.06,54.87,157288.0,14386.0,327274.0,-18.52,58.0,X-LC,3.42,2.0,0.09,2.28,13.26,X40,BTT,IT
76,TMPV,600.00,0.71,-14.14,65.59,42.18,155671.0,-39071.0,237339.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.71,XY24,NTT,AUTO
82,VBL,671.64,-1.02,-9.73,50.29,35.67,143388.0,-30720.0,285122.0,-20.46,41.0,X-LC,6.13,4.0,-0.21,1.99,2.78,X40N,ATH,FMCG
43,ITC,452.00,0.62,-0.32,10.80,10.44,21545.0,-650.0,199488.0,-40.34,50.0,X-LC,0.96,5.0,-0.03,1.39,5.22,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.54,-12.65,107.99,81.67,88489.0,-11868.0,81942.0,7366.67,52.0,L-SC,18.40,271.0,-0.13,0.57,58.68,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.06,-12.13,58.80,39.54,52268.0,-12268.0,88891.0,-31.30,70.0,M-SC,6.90,220.0,-0.23,0.62,25.35,AR,ATH,AUTO
51,MASFIN,398.61,-1.40,-3.89,26.99,22.05,25416.0,-3810.0,94170.0,-17.34,54.0,H-SC,7.00,164.0,-0.15,0.66,36.88,XR,ATH,FINANCE
70,SURYODAY,216.00,2.67,-20.21,54.20,23.03,77442.0,-36189.0,142882.0,55.70,47.0,H-SC,11.65,167.0,-0.47,0.99,41.77,XR,NTT,BANKS
65,SHALBY,327.00,-1.35,-16.14,49.17,25.10,74696.0,-29230.0,151913.0,1016.43,33.0,M-SC,16.66,250.0,-0.39,1.06,21.74,XY24,NTT,HEALTHCARE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-2.15,-9.16,10.11,0.02,16031.0,-15994.0,158565.0,141.63,70.0,M-SC,4.20,234.0,-1.00,1.10,37.92,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.61,0.57,20.26,20.95,51297.0,1426.0,253194.0,-0.49,76.0,X-LC,12.12,31.0,0.03,1.76,36.40,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,2.54,-37.21,111.64,32.89,54121.0,-28728.0,48478.0,-698.49,65.0,L-MC,5.39,259.0,-0.53,0.34,33.51,XR,NTT,BANKS
0,5PAISA,593.00,-2.55,-33.84,70.65,12.90,102132.0,-73936.0,144560.0,112.74,62.0,H-SC,15.05,161.0,-0.72,1.01,20.20,OX40N,NTT,FINANCE
68,SONACOMS,806.63,0.06,-12.13,58.80,39.54,52268.0,-12268.0,88891.0,-31.30,70.0,M-SC,6.90,220.0,-0.23,0.62,25.35,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.73
1,25,44.58
2,50,75.36


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.66
MC    29.27
LC    25.09
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.91
X40      21.11
X40N     11.87
XR        9.66
XY25      9.13
AR        8.99
OX40N     7.70
X200      1.86
SR        0.99
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.07
X-MC    22.84
X-LC    19.95
M-SC    12.04
X-SC     8.14
H-MC     4.71
H-LC     2.97
L-SC     1.41
M-MC     1.38
M-LC     1.19
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.73,-4.87,38.78
IT,13.77,-14.37,73.41
FINANCE,9.47,-14.09,62.29
MISC,7.47,-21.12,70.74
PAINTS,5.99,-7.08,22.99
ELECTRICAL,5.89,-11.13,51.84
INSURANCE,4.44,0.75,34.23
PHARMA,3.84,-1.00,33.33
AUTO,3.35,-19.48,70.71


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3028175.0,21
AR,1280174.0,10
XR,1278446.0,13
X40,1007125.0,14
X40N,745204.0,9
OX40N,684550.0,10
XY25,331539.0,6
SR,277174.0,2
MH,69161.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3519732.0,25
M-SC,1330310.0,15
X-MC,1203013.0,16
X-LC,899063.0,11
X-SC,750443.0,8
H-MC,402497.0,3
L-SC,259555.0,3
H-LC,131811.0,2
M-LC,121201.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1208522.0      6
           AR         867995.0      5
           XR         782210.0      7
M-SC       XY24       777187.0      6
X-MC       X40        435612.0      7
X-LC       X40        392392.0      5
X-MC       XY24       348879.0      3
H-SC       OX40N      314670.0      4
X-SC       X40N       290326.0      3
M-SC       OX40N      282431.0      5
X-SC       XY24       280996.0      3
H-SC       SR         277174.0      2
X-MC       X40N       244664.0      4
X-LC       XY24       225577.0      2
H-MC       AR         215483.0      2
X-LC       X40N       210214.0      2
H-MC       XY24       187014.0      1
X-SC       X40        179121.0      2
X-MC       XY25       173858.0      2
L-SC       XR         172106.0      2
M-SC       XR         148808.0      2
           AR         121884.0      2
M-LC       XR         121201.0      1
L-SC       OX40N       87449.0      1
H-LC       AR          74812.0      1
X-LC       XY25        70880.0      2
H-SC       MH          69161.0      1
H-LC       X200        56999.0      1
L-MC       XR          54121.0      1
M-MC       XY25        47494.0      1
L-LC       XY25        39307.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 61.0 seconds
